In [0]:
a={}
# Retrieve the task value from the previous task (bronze & silver)
bronze_output = dbutils.jobs.taskValues.get(taskKey="Bronze", key="bronze_output",debugValue=a)
silver_data = dbutils.jobs.taskValues.get(taskKey="Silver", key="silver_output",debugValue=a)

# Access individual variables
start_date = bronze_output.get("start_date", "")
silver_adls = bronze_output.get("silver_adls", "")
gold_adls = bronze_output.get("gold_adls", "")

print(f"Start Date: {start_date}, Gold ADLS: {gold_adls}")

Start Date: , Gold ADLS: 


In [0]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# Ensure the below library is installed on your cluster
import reverse_geocoder as rg
from datetime import date, timedelta

In [0]:
silver_file_path=''

In [0]:
#df = spark.read.parquet(silver_output_path).filter(col('time') > start_date)

df = spark.read.parquet(silver_data).filter(col('time') > start_date)


In [0]:
##To speed up processing since our cluster is small
df = df.limit(10)

In [0]:
df.head()

Row(id='ci41026639', longitude=-117.8531667, latitude=36.1331667, elevation=6.55, title='M 0.9 - 13 km NE of Coso Junction, CA', place_description='13 km NE of Coso Junction, CA', sig=13, mag=0.91, magType='ml', time=datetime.datetime(2025, 7, 19, 23, 54, 28, 10000), updated=datetime.datetime(2025, 7, 19, 23, 57, 56, 770000))

In [0]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    try:
        coordinates = (float(lat), float(lon))
        result = rg.search(coordinates)[0].get('cc')
        print(f"Processed coordinates: {coordinates} -> {result}")
        return result
    except Exception as e:
        print(f"Error processing coordinates: {lat}, {lon} -> {str(e)}")
        return None

In [0]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

In [0]:
##testing the functionality

get_country_code(48.8588443, 2.2943506)



Processed coordinates: (48.8588443, 2.2943506) -> FR


'FR'

In [0]:

# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

In [0]:
df_with_location.head(4)

[Row(id='ci41026639', longitude=-117.8531667, latitude=36.1331667, elevation=6.55, title='M 0.9 - 13 km NE of Coso Junction, CA', place_description='13 km NE of Coso Junction, CA', sig=13, mag=0.91, magType='ml', time=datetime.datetime(2025, 7, 19, 23, 54, 28, 10000), updated=datetime.datetime(2025, 7, 19, 23, 57, 56, 770000), country_code='US'),
 Row(id='ak025975facy', longitude=-153.4387, latitude=59.8083, elevation=121.5, title='M 1.6 - 37 km E of Pedro Bay, Alaska', place_description='37 km E of Pedro Bay, Alaska', sig=39, mag=1.6, magType='ml', time=datetime.datetime(2025, 7, 19, 23, 41, 56, 20000), updated=datetime.datetime(2025, 7, 19, 23, 44, 22, 697000), country_code='US'),
 Row(id='tx2025odcpdo', longitude=-102.171, latitude=32.356, elevation=4.054, title='M 1.5 - 34 km E of McKinney Acres, Texas', place_description='34 km E of McKinney Acres, Texas', sig=35, mag=1.5, magType='ml', time=datetime.datetime(2025, 7, 19, 23, 40, 30, 560000), updated=datetime.datetime(2025, 7, 20,

In [0]:
df_with_location.display()

id,longitude,latitude,elevation,title,place_description,sig,mag,magType,time,updated,country_code
ci41026639,-117.8531667,36.1331667,6.55,"M 0.9 - 13 km NE of Coso Junction, CA","13 km NE of Coso Junction, CA",13,0.91,ml,2025-07-19T23:54:28.01Z,2025-07-19T23:57:56.77Z,US
ak025975facy,-153.4387,59.8083,121.5,"M 1.6 - 37 km E of Pedro Bay, Alaska","37 km E of Pedro Bay, Alaska",39,1.6,ml,2025-07-19T23:41:56.02Z,2025-07-19T23:44:22.697Z,US
tx2025odcpdo,-102.171,32.356,4.054,"M 1.5 - 34 km E of McKinney Acres, Texas","34 km E of McKinney Acres, Texas",35,1.5,ml,2025-07-19T23:40:30.56Z,2025-07-20T00:00:39.272Z,US
hv74730067,-155.484832763672,19.2106666564941,29.6700000762939,"M 2.0 - 1 km NW of Pāhala, Hawaii","1 km NW of Pāhala, Hawaii",61,1.99,ml,2025-07-19T23:38:48.65Z,2025-07-19T23:42:13.26Z,US
ak025975dbm6,-153.1983,58.6548,84.7,"M 1.7 - 68 km NNW of Aleneva, Alaska","68 km NNW of Aleneva, Alaska",44,1.7,ml,2025-07-19T23:32:39.182Z,2025-07-19T23:35:04.671Z,US
ci41026623,-117.2143333,34.011,3.03,"M 1.4 - 6 km SSW of Redlands, CA","6 km SSW of Redlands, CA",31,1.43,ml,2025-07-19T23:27:33.31Z,2025-07-19T23:31:00.223Z,US
ak025975bzgh,-152.1653,61.9274,117.4,"M 1.6 - 40 km W of Skwentna, Alaska","40 km W of Skwentna, Alaska",39,1.6,ml,2025-07-19T23:26:15.058Z,2025-07-19T23:28:27.335Z,US
ak0259759m7g,-152.3056,61.0712,114.0,"M 1.4 - 63 km W of Tyonek, Alaska","63 km W of Tyonek, Alaska",30,1.4,ml,2025-07-19T23:15:10.201Z,2025-07-19T23:17:41.962Z,US
tx2025odbooa,-104.589,31.598,2.8122,"M 1.9 - 65 km NNE of Van Horn, Texas","65 km NNE of Van Horn, Texas",56,1.9,mlv,2025-07-19T23:10:27.226Z,2025-07-20T05:12:33.425Z,US
nc75212237,-122.767166137695,38.8269996643066,2.29999995231628,"M 0.8 - 4 km WNW of Cobb, CA","4 km WNW of Cobb, CA",9,0.76,md,2025-07-19T23:09:48.79Z,2025-07-19T23:11:22.721Z,US


In [0]:
# adding significance classification field
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

In [0]:

df_with_location_sig_class.display()

id,longitude,latitude,elevation,title,place_description,sig,mag,magType,time,updated,country_code,sig_class
ci41026639,-117.8531667,36.1331667,6.55,"M 0.9 - 13 km NE of Coso Junction, CA","13 km NE of Coso Junction, CA",13,0.91,ml,2025-07-19T23:54:28.01Z,2025-07-19T23:57:56.77Z,US,Low
ak025975facy,-153.4387,59.8083,121.5,"M 1.6 - 37 km E of Pedro Bay, Alaska","37 km E of Pedro Bay, Alaska",39,1.6,ml,2025-07-19T23:41:56.02Z,2025-07-19T23:44:22.697Z,US,Low
tx2025odcpdo,-102.171,32.356,4.054,"M 1.5 - 34 km E of McKinney Acres, Texas","34 km E of McKinney Acres, Texas",35,1.5,ml,2025-07-19T23:40:30.56Z,2025-07-20T00:00:39.272Z,US,Low
hv74730067,-155.484832763672,19.2106666564941,29.6700000762939,"M 2.0 - 1 km NW of Pāhala, Hawaii","1 km NW of Pāhala, Hawaii",61,1.99,ml,2025-07-19T23:38:48.65Z,2025-07-19T23:42:13.26Z,US,Low
ak025975dbm6,-153.1983,58.6548,84.7,"M 1.7 - 68 km NNW of Aleneva, Alaska","68 km NNW of Aleneva, Alaska",44,1.7,ml,2025-07-19T23:32:39.182Z,2025-07-19T23:35:04.671Z,US,Low
ci41026623,-117.2143333,34.011,3.03,"M 1.4 - 6 km SSW of Redlands, CA","6 km SSW of Redlands, CA",31,1.43,ml,2025-07-19T23:27:33.31Z,2025-07-19T23:31:00.223Z,US,Low
ak025975bzgh,-152.1653,61.9274,117.4,"M 1.6 - 40 km W of Skwentna, Alaska","40 km W of Skwentna, Alaska",39,1.6,ml,2025-07-19T23:26:15.058Z,2025-07-19T23:28:27.335Z,US,Low
ak0259759m7g,-152.3056,61.0712,114.0,"M 1.4 - 63 km W of Tyonek, Alaska","63 km W of Tyonek, Alaska",30,1.4,ml,2025-07-19T23:15:10.201Z,2025-07-19T23:17:41.962Z,US,Low
tx2025odbooa,-104.589,31.598,2.8122,"M 1.9 - 65 km NNE of Van Horn, Texas","65 km NNE of Van Horn, Texas",56,1.9,mlv,2025-07-19T23:10:27.226Z,2025-07-20T05:12:33.425Z,US,Low
nc75212237,-122.767166137695,38.8269996643066,2.29999995231628,"M 0.8 - 4 km WNW of Cobb, CA","4 km WNW of Cobb, CA",9,0.76,md,2025-07-19T23:09:48.79Z,2025-07-19T23:11:22.721Z,US,Low


In [0]:

# Save the transformed DataFrame to the Silver container
#gold_output_path = f"abfss://gold@earthquakedatafiles.dfs.core.windows.net/earthquake_events_gold/"
#print (gold_output_path)

# Save the transformed DataFrame to the Silver container
gold_output_path = f"{gold_adls}earthquake_events_gold/"

abfss://gold@earthquakedatafiles.dfs.core.windows.net/earthquake_events_gold/


In [0]:
# Append DataFrame to gold container in Parquet format
df_with_location_sig_class.write.mode('append').parquet(gold_output_path)